In [ ]:
import sys
sys.path.append("\..")

from lava.proc.lif.process import LIF
from lava.lib.dnf.connect.connect import connect
from lava.lib.dnf.operations.operations import Weights
from lava.lib.dnf.kernels.kernels import MultiPeakKernel
from lava.lib.dnf.operations.operations import Convolution
from lava.lib.dnf.kernels.kernels import SelectiveKernel
from lava.proc.io.source import RingBuffer

from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.inputs.rate_code_spike_gen.process import RateCodeSpikeGen
from lava.proc.monitor.process import Monitor
import scipy.sparse


import matplotlib.pyplot as plt

import numpy as np

In [ ]:
k_size=5
old_shape = np.array([180, 240])
img_shape = (old_shape / k_size).astype(int)
depthsxmeter = 10
d_range = [0.0, 3.0]

n_depths = int(d_range[1] - d_range[0]) * depthsxmeter
shape = (img_shape[0], img_shape[1], n_depths)
print(shape)

In [ ]:
def subsamples(data, k_size):
    new_shape = (
        int(data.shape[0] / k_size),
        k_size,
        int(data.shape[1] / k_size),
        k_size)
    data_reshape = data.reshape(new_shape)
    
    return np.nanmean(data_reshape.swapaxes(1 , 2), axis=(-1, -2))

In [ ]:
dnf = LIF(shape=shape, du=0.12, dv=0.5, vth=10)

# kernel = SelectiveKernel(amp_exc=4.0,
#                          width_exc=20.0,
#                          global_inh=-3)
kernel = MultiPeakKernel(amp_exc=7,
                         width_exc=[2, 2, 5],
                         amp_inh=-125,
                         width_inh=[0.1, 0.1, 10])
connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])
# connect(input_proc.s_out, dnf.a_in, [Weights(7)])

In [ ]:
time_steps = 100

# input_buffer = np.zeros((shape[0], shape[1], shape[2], time_steps))
# input_buffer[5, 5, 5, :] = 1
# print(input_buffer.shape)

samples = np.load("samples_slider.npy", allow_pickle=True)

samples_np = np.zeros((180, 240, samples.size))

for i, s in enumerate(samples):
    samples_np[:,:, i] = s.toarray()
    
input_buffer = np.zeros((shape[0], shape[1], shape[2], time_steps))
samp = samples_np[:,:, 1000:1100:1]

for i in range(samp.shape[-1]):
    test = samp[:,:,i]
    test[test == 0] = np.nan
    
    test_sub = subsamples(test, k_size)
    
    data_s = scipy.sparse.csr_matrix(test_sub[:])

    y, x, d = scipy.sparse.find(data_s)
    y = y[d<d_range[1]]
    x = x[d<d_range[1]]
    d = d[d<d_range[1]]
    d = (((d - d_range[0]) / (d_range[1] - d_range[0])) * n_depths).astype(int)
    input_buffer[y, x, d, i] = 1

print(input_buffer.shape)
input_proc = RingBuffer(input_buffer)

In [ ]:
connect(input_proc.s_out, dnf.a_in, [Weights(7)])

In [ ]:

monitor = Monitor()
monitor.probe(dnf.s_out, time_steps)

# monitor_v = Monitor()
# monitor_v.probe(dnf.v, time_steps)

# monitor_u = Monitor()
# monitor_u.probe(dnf.u, time_steps)

# Run the DNF
dnf.run(condition=RunSteps(num_steps=time_steps),
        run_cfg=Loihi1SimCfg(select_tag='floating_pt'))

# Get probed data from monitor
probed_data = monitor.get_data()
# probed_data_v = monitor_v.get_data()
# probed_data_u = monitor_u.get_data()

# Stop the execution after getting the monitor's data
dnf.stop()

In [ ]:
t = probed_data[dnf.name][dnf.s_out.name][80:100,:,:,:].sum(axis=(0))
print(t.shape)

In [ ]:
# t = input_buffer[:, :, :, 80:100].sum(axis=-1)

In [ ]:
%matplotlib notebook
ax = plt.figure().add_subplot(projection='3d')
ax.voxels(t);

In [ ]:
v = np.zeros_like(t)

In [ ]:
for i in range(v.shape[-1]):
    v[:, :, i] = (i / n_depths) * (d_range[1] - d_range[0]) + d_range[0]

In [ ]:
test = t * v

In [ ]:
test.shape

In [ ]:
%matplotlib inline
test[test == 0] = np.nan
m = plt.imshow(np.nanmean(test, axis=-1), interpolation="none", cmap="jet_r", vmax=2.5, vmin=0)
plt.colorbar(m)

In [ ]:
plt.hist(test.flatten(), 30, range=(0, 3));